<a href="https://colab.research.google.com/gist/rmeziatisab/20820a7c8cc667a1da44f22bcbcb7923/pose_detection_mediapipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human pose detection using MediaPipe
MediaPipe is a powerful set of tools developed by Google, allowing to apply diversified tasks linked to computer vision (*eg* object detection, gesture recognition), text (*eg* text embedding) and audio data (audio classification). In this tutorial, I am going to show you how to use MediaPipe to detect pose landmarks in videos, following these steps:


*   Import and install the necessary packages
*   Download the MediaPipe pose landmark detection model
*   Load an input video
*   Extract the pose landmarks on each video frame, and visulize them
*   Save the current frame alongside with the extracted pose landmarks



## Required packages

To be able to follow this notebook, the following packages should be installed:


*   MediaPipe
*   OpenCV
*   NumPy

This tuto does not cover opencv and numpy installation. You can get MediaPipe installed using:



In [ ]:
# Install MediaPipe
!pip install -q mediapipe==0.10.0

Let's begin with importing the above-mentioned packages:

In [ ]:
# Import the required packages
import cv2
import numpy as np
import mediapipe as mp

If you are running this notebook on Google Colab, and need to access files in you Google Drive, you will need to mount to Google Drive. This can be achieved using:

In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Pose landmark detection model

The pose landmarker model proposed by MediaPipe allows to extract 33 body landmark 3D coordinates, estimate for each landmark whether it is visible in the input frame (meaning not hided by another body part or an object), and indicate the probabibility that it is present in the frame.

You can download the pose landmarker model [here](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker#models). For this tutorial, I will use the Full model. Here is another option to get the pose landmarker model:

In [ ]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_full.task

## Pose landmark visualization
For body landmark visualization, I will be using the following function, developed by the MediaPipe authors. This function will be called to annotate the current frame with the detected body landmarks, as well as to draw connections between them.

In [ ]:
# Use function allowing to draw pose landmarks on an image
from mediapipe.python import solutions
from mediapipe.framework.formats import landmark_pb2

def draw_landmarks_on_image(rgb_image, detection_result):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]

        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            pose_landmarks_proto,
            solutions.pose.POSE_CONNECTIONS,
            solutions.drawing_styles.get_default_pose_landmarks_style()
        )
    return annotated_image

## Pose landmark extraction
The first thing to do is load the video MediaPipe pose detection model will be applied to. The frame rate (expressed in frames per second, fps) and the frame dimensions are used to create the ouput video that will contain the input frames annotated with the detected pose landmarks.

In [ ]:
# Read the input video and extract video information
input_path = '/content/drive/My Drive/MOCA/Code/PoseEstimation/mini_vid.mp4'#'/content/drive/My Drive/MOCA/Data/YT-videos/cropped/sidehop00.mp4'
cap = cv2.VideoCapture(input_path)

if cap.isOpened() is False:
    print('Video not found')
else:
  fps = cap.get(cv2.CAP_PROP_FPS)
  width = int(cap.get(cv2. CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2. CAP_PROP_FRAME_HEIGHT))

We need to create a VideoWriter object to be able to save the output video. The video path - containing the video file extension - should be specified. The VideoWriter object constructor takes the path, the FourCC, the frame rate as well as the frame size. An `'MP4V'` FourCC corresponds to the MP4 format.

In [ ]:
# Create a VideoWriter object
output_path = '/content/drive/My Drive/MOCA/Code/PoseEstimation/test.mp4'
output_vid = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))

Next, we set the options needed to create a PoseLandmarked object. Among the options, the downloaded model path and the video mode are indicated.

In [ ]:
# Set options for the PoseLandmarker object
from mediapipe.tasks import python

model_path = '/content/drive/My Drive/MOCA/Code/PoseEstimation/mediaPipe/pose_landmarker_full.task'

BaseOptions = python.BaseOptions
PoseLandmarker = python.vision.PoseLandmarker
PoseLandmarkerOptions = python.vision.PoseLandmarkerOptions
VisionRunningMode = python.vision.RunningMode
options = PoseLandmarkerOptions(base_options=BaseOptions(model_asset_path=model_path),
                                running_mode=VisionRunningMode.VIDEO)

Pose landmarks are detected for the input video frames: each frame is converted to an Image object that is passed to the `detect_for_video` method, with its corresponding frame timestamp obtained as:

<center> $ frame\,timestamp = \frac{1000\,*\,frame\,index}{frame\,rate} $ </center>

The frame index is an integer between $0$ and the total number of frames, describing the current frame number. The frame index can be obtained either with a counter variable incremented each time a new frame is read, or using the VideoCapure `get` method as follows:
`int(cap.get(cv2.CAP_PROP_POS_FRAMES))`.

Then, the `draw_landmarks_on_image` function is called to draw the detected body landmark positions and connections. I use the `cv2_imshow` function to display the annotated frames since I am using Google Colab. If you are using another environment, you can use `cv2.imshow`.

Finally, the annotated frame is saved as an output video frame.


In [ ]:
from google.colab.patches import cv2_imshow

# Extract pose landmarks
with PoseLandmarker.create_from_options(options) as landmarker:
    frame_index = 0
    while cap.isOpened():
        hasFrame, image = cap.read()
        if not hasFrame:
            print('No more frames to read!')
            break

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        numpy_frame_from_opencv = np.asarray(image)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=numpy_frame_from_opencv)
        cv_frame_index = cap.get(cv2.CAP_PROP_POS_FRAMES)
        frame_index += 1
        frame_timestamp_ms = int(1000*frame_index / fps)
        result = landmarker.detect_for_video(mp_image, frame_timestamp_ms)
        annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), result)
        cv_image = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
        cv2_imshow(cv_image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

        output_vid.write(cv_image)

    cap.release()
    output_vid.release()

cv2.destroyAllWindows()